In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import re
from unidecode import unidecode

tqdm.pandas()

In [ ]:
page = requests.get(
    "https://thefangirlverdict.com/index/all-reviews/full-list-of-shows/"
)
soup = BeautifulSoup(page.content)

In [ ]:
for p in soup.find_all("p", style="text-align:justify"):
    p.decompose()

In [ ]:
names = []
urls = []
grades = []

for x in tqdm(range(1200)):

    element = soup.select_one("#post-7658 > div > p:nth-child(" + str(x) + ")")
    if element is None:
        continue

    kdrama = element.findChildren("a", recursive=False)
    # print(kdrama)
    grade = element.findChildren("strong", recursive=False)
    # print(grade)

    if kdrama:
        names.append(kdrama[0].contents[0])
        urls.append(kdrama[0].get("href"))
        if grade:
            grades.append(grade[0].text)
        else:
            grades.append("NA")

In [ ]:
df = pd.DataFrame({"source": names, "url": urls, "rating": grades})
df.info()

In [ ]:
df.head()

In [ ]:
# df['rating'] = df['rating'].replace(['A++', 'A+', 'A', 'A-','B++','B+','B','B-','C++', 'C+', 'C', 'C-','D++','D+','D','D-'],['A1', 'A2', 'A3', 'A4','B1','B2','B3','B4','C1', 'C2', 'C3', 'C4','D1','D2','D3','D4'])

In [ ]:
# handle NAs
df = df.replace("NA", np.NaN)
df.dropna(inplace=True)

In [ ]:
df["year"] = df.apply(lambda row: re.search(r"\d+", row["url"]).group(), axis=1)
# df["source"] = df.apply(lambda row: unidecode(row["source"]), axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv("df.csv", encoding="utf-8", index=False)